# TRANSFORMER
Per implementare una backbone transformer su un modello R-CNN di Detectron2, sfrutto l’architettura base di modelli come Vision Transformer (ViT) o Swin Transformer integrati come backbone. Detectron2 supporta nativamente il caricamento di backbone personalizzati, purché siano definiti in conformità con l’architettura e i requisiti del framework.

### 1. Installare Dipendenze
Se il transformer che vuoi utilizzare non è integrato in Detectron2, puoi installare una libreria che fornisce implementazioni, come timm o transformers.<br>
**pip install timm**

### 2. Configurazione del Modello
Aggiorna il file di configurazione del modello Detectron2 per utilizzare il tuo transformer come backbone:

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from swin_transformer_detectron2 import add_swin_transformer_config

# Crea configurazione
cfg = get_cfg()

# Aggiungi configurazione per Swin Transformer
add_swin_transformer_config(cfg)

# Carica una configurazione di base per Faster R-CNN
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Modifica il backbone con Swin Transformer
cfg.MODEL.BACKBONE.NAME = "build_swin_transformer_backbone"
cfg.MODEL.SWIN = {
    "PRETRAIN_IMG_SIZE": 224,
    "PATCH_SIZE": 4,
    "DEPTHS": [2, 2, 6, 2],
    "NUM_HEADS": [3, 6, 12, 24],
    "WINDOW_SIZE": 7,
    "MLP_RATIO": 4.0,
    "DROP_PATH_RATE": 0.2,
}

# Configura i pesi pre-addestrati per Swin
cfg.MODEL.WEIGHTS = "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth"

# Configura il dispositivo e le classi
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80  # Modifica in base al tuo dataset
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Salva la configurazione
cfg.OUTPUT_DIR = "./output_swin_transformer"

### 3. Costruzione Modello

In [ ]:
from detectron2.modeling import build_model

# Costruisci il modello con il backbone Swin
model = build_model(cfg)

# Visualizza la struttura del modello
print(model)

### 4. Inference

In [ ]:
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
import cv2

# Configura il predittore
predictor = DefaultPredictor(cfg)

# Carica un'immagine di test
image = cv2.imread("test_image.jpg")

# Effettua inferenza
outputs = predictor(image)

# Visualizza i risultati
v = Visualizer(image[:, :, ::-1], metadata=None)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow("Results", out.get_image()[:, :, ::-1])
cv2.waitKey(0)